# Resampling and Normalization

In [11]:
import pandas as pd
import librosa
import soundfile as sf
import os
import numpy as np

In [ ]:


input_csv = '../data/cleaned/70_15_15_cleaned_train.csv'
input_audio_dir = '../data/raw/audio/xeno_canto'
output_audio_dir = '../data/processed/audio/xeno_canto'


os.makedirs(output_audio_dir, exist_ok=True)


sample_rate = 16000  # Set sample rate the same as used in the pretrained model
target_db_level = -20  


df = pd.read_csv(input_csv)


for index, row in df.iterrows():
    audio_id = row['id']
    

    audio_path_pattern = os.path.join(input_audio_dir, f"{audio_id}.*")
    audio_files = glob.glob(audio_path_pattern)
    
    if len(audio_files) == 0:
        print(f"No audio file found for ID {audio_id}. Skipping.")
        continue
    elif len(audio_files) > 1:
        print(f"Multiple audio files found for ID {audio_id}. Skipping to avoid ambiguity.")
        continue
    
    audio_path = audio_files[0]
    
    try:
        y, sr = librosa.load(audio_path, sr=None)
        
        if sr != sample_rate:
            y = librosa.resample(y, orig_sr=sr, target_sr=sample_rate)
            sr = sample_rate
        
        rms = librosa.feature.rms(y=y)[0]
        current_db = librosa.amplitude_to_db(rms, ref=np.max)
        db_adjustment = target_db_level - np.mean(current_db)
        y = y * (10 ** (db_adjustment / 20))

        output_path = os.path.join(output_audio_dir, f"{audio_id}.wav")
        sf.write(output_path, y, sr)
        
        print(f"Processed {audio_id} successfully.")
        
    except Exception as e:
        print(f"Error processing {audio_id}: {e}")

print("Processing complete.")

Processed 370848 successfully.
Processed 629154 successfully.
Processed 579339 successfully.
Processed 802334 successfully.
Processed 799697 successfully.
Processed 723673 successfully.
Processed 842246 successfully.
Processed 384576 successfully.
Processed 892221 successfully.


Processed 411273 successfully.
Processed 170657 successfully.
Processed 83186 successfully.
Processed 662223 successfully.
Processed 812133 successfully.
Processed 721177 successfully.
Processed 753957 successfully.
Processed 548431 successfully.
Processed 357501 successfully.
Processed 766676 successfully.
Processed 800530 successfully.
Processed 904183 successfully.
Processed 438364 successfully.
Processed 687646 successfully.
Processed 673499 successfully.
Processed 407938 successfully.
Processed 501169 successfully.


Processed 923199 successfully.
Processed 467130 successfully.
Processed 596933 successfully.
Processed 549876 successfully.
Processed 315401 successfully.


Processed 315498 successfully.
Processed 431457 successfully.
Processed 929636 successfully.
Processed 692261 successfully.
Processed 508014 successfully.
Processed 667816 successfully.
Processed 522337 successfully.
Processed 696627 successfully.
Processed 690184 successfully.


Processed 368779 successfully.
Processed 606005 successfully.
Processed 922908 successfully.
Processed 600138 successfully.
Processed 794153 successfully.
Processed 672738 successfully.
Processed 676352 successfully.
Processed 608129 successfully.
Processed 547009 successfully.
Processed 912948 successfully.
Processed 557017 successfully.
Processed 356547 successfully.
Processed 922375 successfully.
Processed 677280 successfully.
Processed 435163 successfully.
Processed 800000 successfully.
Processed 780672 successfully.
Processed 600469 successfully.
Processed 921930 successfully.
Processed 158103 successfully.
Processed 720022 successfully.


...

Processed 662760 successfully.
Processed 734037 successfully.
Processed 915489 successfully.
Processed 606400 successfully.
Processed 834367 successfully.
Processed 759610 successfully.
Processed 328500 successfully.
Processed 579939 successfully.
Processed 829642 successfully.
Processed 240704 successfully.
Processed 566327 successfully.
Processed 785661 successfully.


Processed 927439 successfully.
Processed 651566 successfully.
Processed 710154 successfully.
Processed 875526 successfully.
Processed 696592 successfully.
Processed 110343 successfully.
Processed 808347 successfully.
Processed 839965 successfully.
Processed 799893 successfully.
Processed 478071 successfully.
Processed 792884 successfully.
Processed 345723 successfully.
Processed 655214 successfully.
Processed 591417 successfully.
Processed 667882 successfully.
Processed 24829 successfully.
Processed 635135 successfully.


Processed 692026 successfully.
Processed 930069 successfully.
Processed 690300 successfully.
Processed 558971 successfully.


Processed 555039 successfully.
Processed 735078 successfully.
Processed 767845 successfully.
Processed 483916 successfully.
Processed 600205 successfully.
Processed 641855 successfully.
Processed 499155 successfully.
Processed 496377 successfully.
Processed 599176 successfully.
Processed 679985 successfully.
Processed 593284 successfully.
Processed 28213 successfully.


Processed 901563 successfully.
Processed 639789 successfully.
Processed 788372 successfully.
Processed 602367 successfully.
Processed 346300 successfully.
Processed 924397 successfully.
Processed 805777 successfully.
Processed 750147 successfully.
Processed 668274 successfully.
Processed 263949 successfully.
Processed 804637 successfully.
Processed 331148 successfully.
Processed 214155 successfully.


# Trim and Segment Audio

In [12]:
input_audio_dir = '../data/processed/audio/xeno_canto'  # Assuming previous output directory as input here
output_audio_dir = '../data/processed/segmented_audio/xeno_canto'

os.makedirs(output_audio_dir, exist_ok=True)


sample_rate = 16000  
segment_duration = 5  # Segment duration in seconds
overlap_duration = 1  # Overlap duration in seconds
target_db_level = -20  

df = pd.read_csv(input_csv)

for index, row in df.iterrows():
    audio_id = row['id']
    audio_path = os.path.join(input_audio_dir, f"{audio_id}.wav")
    
    try:
        y, sr = librosa.load(audio_path, sr=sample_rate)
        
        # Trim silence from the beginning and end of the audio
        y, _ = librosa.effects.trim(y, top_db=20) 

        # Normalize the audio to target dB level
        rms = librosa.feature.rms(y=y)[0]
        current_db = librosa.amplitude_to_db(rms, ref=np.max)
        db_adjustment = target_db_level - np.mean(current_db)
        y = y * (10 ** (db_adjustment / 20))

        # Calculate segment and overlap lengths in samples
        segment_length = segment_duration * sr
        overlap_length = overlap_duration * sr
        step = segment_length - overlap_length  

        # Split audio into overlapping segments
        total_segments = int(np.ceil((len(y) - overlap_length) / step))

        for i in range(total_segments):
            start = i * step
            end = min(start + segment_length, len(y))
            segment = y[start:end]

            if len(segment) < segment_length and end != len(y):
                continue
            
            segment_filename = f"{audio_id}_segment_{i+1}.wav"
            output_path = os.path.join(output_audio_dir, segment_filename)
            sf.write(output_path, segment, sr)
            
            print(f"Processed segment {i+1} of {audio_id} successfully.")
        
    except Exception as e:
        print(f"Error processing {audio_id}: {e}")

print("Audio trimming and segmentation complete with overlapping segments.")


Processed segment 1 of 370848 successfully.
Processed segment 2 of 370848 successfully.
Processed segment 3 of 370848 successfully.
Processed segment 1 of 629154 successfully.
Processed segment 2 of 629154 successfully.
Processed segment 3 of 629154 successfully.
Processed segment 1 of 579339 successfully.
Processed segment 2 of 579339 successfully.
Processed segment 1 of 802334 successfully.
Processed segment 1 of 799697 successfully.
Processed segment 2 of 799697 successfully.
Processed segment 1 of 723673 successfully.
Processed segment 2 of 723673 successfully.
Processed segment 1 of 842246 successfully.
Processed segment 1 of 384576 successfully.
Processed segment 2 of 384576 successfully.
Processed segment 1 of 892221 successfully.
Processed segment 2 of 892221 successfully.
Processed segment 1 of 411273 successfully.
Processed segment 2 of 411273 successfully.
Processed segment 3 of 411273 successfully.
Processed segment 1 of 170657 successfully.
Processed segment 2 of 170657 su

/tmp/ipykernel_1217381/1961227399.py:19: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=sample_rate)
/home/maiko/anaconda3/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processed segment 2 of 201340 successfully.
Processed segment 1 of 671486 successfully.
Processed segment 1 of 708215 successfully.
Processed segment 2 of 708215 successfully.
Processed segment 1 of 687010 successfully.
Processed segment 2 of 687010 successfully.
Processed segment 1 of 678763 successfully.
Processed segment 2 of 678763 successfully.
Processed segment 1 of 678899 successfully.
Processed segment 1 of 470723 successfully.
Processed segment 2 of 470723 successfully.
Processed segment 1 of 345752 successfully.
Processed segment 2 of 345752 successfully.
Processed segment 1 of 507023 successfully.
Processed segment 1 of 552362 successfully.
Processed segment 2 of 552362 successfully.
Processed segment 3 of 552362 successfully.
Processed segment 1 of 435198 successfully.
Processed segment 1 of 931620 successfully.
Processed segment 2 of 931620 successfully.
Processed segment 1 of 878100 successfully.
Processed segment 2 of 878100 successfully.
Processed segment 1 of 574639 su